Notebook header + imports + version logging

In [3]:
# [CELL 02-00] Notebook 02 — EDA Target MOOC (MARS)
# Goal: load raw target dataset (MARS), inspect schema, compute EDA counts, and export canonical events parquet.
# Canonical schema (agreed): user_id, item_id, timestamp, signal_type, value
# Guardrail: NO session-gap analysis, NO sessionization, NO prefix->label samples.

import os
import sys
import json
import math
import time
import platform
from pathlib import Path
from datetime import datetime

import pandas as pd

try:
    import duckdb
except Exception as e:
    raise RuntimeError("duckdb is required for this notebook. Please install duckdb.") from e

print("[02-00] Python:", sys.version)
print("[02-00] Platform:", platform.platform())
print("[02-00] pandas:", pd.__version__)
print("[02-00] duckdb:", duckdb.__version__)
print("[02-00] CWD:", Path.cwd().resolve())


[02-00] Python: 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)]
[02-00] Platform: Windows-10-10.0.22621-SP0
[02-00] pandas: 2.3.3
[02-00] duckdb: 1.4.3
[02-00] CWD: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\notebooks


Locate repo root (Windows-safe) + standard folders

In [4]:
# [CELL 02-01] Bootstrap: locate repo root reliably (Windows-safe)

from pathlib import Path

CWD = Path.cwd().resolve()
print("[02-01] Initial CWD:", CWD)

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "PROJECT_STATE.md").exists():
            print(f"[02-01] Found PROJECT_STATE.md in: {p}")
            return p
    for p in [start, *start.parents]:
        if (p / ".git").exists():
            print(f"[02-01] Found .git in: {p}")
            return p
    raise FileNotFoundError("Could not locate repo root (PROJECT_STATE.md or .git not found).")

REPO_ROOT = find_repo_root(CWD)
print("[02-01] REPO_ROOT:", REPO_ROOT)

NOTEBOOKS_DIR = REPO_ROOT / "notebooks"
DATA_DIR      = REPO_ROOT / "data"
RAW_DIR       = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
REPORTS_DIR   = REPO_ROOT / "reports"

for d in [DATA_DIR, RAW_DIR, PROCESSED_DIR, REPORTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("[02-01] RAW_DIR:", RAW_DIR)
print("[02-01] PROCESSED_DIR:", PROCESSED_DIR)
print("[02-01] REPORTS_DIR:", REPORTS_DIR)


[02-01] Initial CWD: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\notebooks
[02-01] Found PROJECT_STATE.md in: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta
[02-01] REPO_ROOT: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta
[02-01] RAW_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw
[02-01] PROCESSED_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed
[02-01] REPORTS_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports


Create RUN_TAG + run folders + helpers

In [5]:
# [CELL 02-02] Run tag + output folders + helpers

RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_REPORT_DIR = REPORTS_DIR / "02_eda_target_mooc" / RUN_TAG
RUN_REPORT_DIR.mkdir(parents=True, exist_ok=True)

TARGET_NAME = "mars"
TARGET_PROCESSED_DIR = PROCESSED_DIR / TARGET_NAME
TARGET_PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("[02-02] RUN_TAG:", RUN_TAG)
print("[02-02] RUN_REPORT_DIR:", RUN_REPORT_DIR)
print("[02-02] TARGET_PROCESSED_DIR:", TARGET_PROCESSED_DIR)

def save_json(path: Path, obj: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2, ensure_ascii=False), encoding="utf-8")
    print("[02-02] Wrote JSON:", path)

def save_df_csv(path: Path, df: pd.DataFrame, index=False):
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=index)
    print("[02-02] Wrote CSV:", path, "| shape:", df.shape)

run_meta = {
    "run_tag": RUN_TAG,
    "notebook": "02_eda_target_mooc.ipynb",
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "repo_root": str(REPO_ROOT),
    "raw_dir": str(RAW_DIR),
    "processed_dir": str(PROCESSED_DIR),
    "target_processed_dir": str(TARGET_PROCESSED_DIR),
    "python": sys.version,
    "pandas": pd.__version__,
    "duckdb": duckdb.__version__,
    "platform": platform.platform(),
}
save_json(RUN_REPORT_DIR / "run_meta.json", run_meta)


[02-02] RUN_TAG: 20251229_131137
[02-02] RUN_REPORT_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137
[02-02] TARGET_PROCESSED_DIR: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\mars
[02-02] Wrote JSON: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\run_meta.json


Locate MARS raw files (auto-discover; fail loudly if missing)

In [6]:
# [CELL 02-03] Discover MARS raw files (EXPLICIT ONLY)

MARS_RAW_DIR = RAW_DIR / "mars"
if not MARS_RAW_DIR.exists():
    raise FileNotFoundError(f"Missing MARS folder: {MARS_RAW_DIR}")

explicit_csv = MARS_RAW_DIR / "explicit_ratings_en.csv"
print("[02-03] MARS dir:", MARS_RAW_DIR)
print("[02-03] explicit exists:", explicit_csv.exists(), "|", explicit_csv)

if not explicit_csv.exists():
    raise FileNotFoundError("Missing explicit_ratings_en.csv under data/raw/mars/")

# Optional context tables (not used here, but logged)
items_csv = MARS_RAW_DIR / "items_en.csv"
users_csv = MARS_RAW_DIR / "users_en.csv"
print("[02-03] items exists:", items_csv.exists(), "|", items_csv)
print("[02-03] users exists:", users_csv.exists(), "|", users_csv)


[02-03] MARS dir: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\mars
[02-03] explicit exists: True | D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\mars\explicit_ratings_en.csv
[02-03] items exists: True | D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\mars\items_en.csv
[02-03] users exists: True | D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\mars\users_en.csv


Validate file types (must be consistent)

In [8]:
# [CELL 02-04] Confirm file formats (expected CSV)

assert explicit_csv.suffix.lower() == ".csv"
print("[02-04] OK: MARS interaction files are CSV.")


[02-04] OK: MARS interaction files are CSV.


DuckDB: build a raw view (no memory blowups)

In [ ]:
# [CELL 02-05] Build DuckDB raw view for MARS (EXPLICIT-ONLY by design)
# 
# DECISION: Use explicit_ratings_en.csv ONLY
# REASON: Focus on explicit preference signals for this research
# NOTE: implicit_ratings_en.csv exists but is deliberately NOT used

con = duckdb.connect(database=":memory:")
con.execute("PRAGMA threads=8;")

RAW_VIEW = "raw_target"

explicit_path = str(explicit_csv.resolve())
print("[02-05] Loading EXPLICIT ratings only (by design):", explicit_path)
print("[02-05] NOTE: implicit_ratings_en.csv is NOT used in this analysis")

con.execute(f"""
CREATE OR REPLACE VIEW {RAW_VIEW} AS
SELECT * FROM read_csv_auto('{explicit_path}', header=True);
""")

schema_df = con.execute(f"DESCRIBE SELECT * FROM {RAW_VIEW}").df()
print("[02-05] Raw schema:")
display(schema_df)

sample_df = con.execute(f"SELECT * FROM {RAW_VIEW} LIMIT 5").df()
print("[02-05] Sample rows:")
display(sample_df)

[02-05] explicit: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\mars\explicit_ratings_en.csv
[02-05] Raw schema:


,column_name,column_type,null,key,default,extra
0,user_id,BIGINT,YES,None,None,None
1,item_id,BIGINT,YES,None,None,None
2,watch_percentage,BIGINT,YES,None,None,None
3,created_at,TIMESTAMP,YES,None,None,None
4,rating,BIGINT,YES,None,None,None


[02-05] Sample rows:


,user_id,item_id,watch_percentage,created_at,rating
0,224557,510,100,2018-09-28 16:18:29,10
1,224557,615,100,2018-09-28 16:22:22,10
2,224557,7680,100,2018-09-28 16:23:34,10
3,224293,510,100,2018-09-28 17:20:30,10
4,224293,515,100,2018-09-28 17:40:02,10


In [10]:
# Diagnostic (standalone): inspect implicit_ratings_en.csv with pandas
import pandas as pd
from pathlib import Path

implicit_csv = Path(RAW_DIR) / "mars" / "implicit_ratings_en.csv"
print("Implicit CSV:", implicit_csv)

df_imp = pd.read_csv(implicit_csv, nrows=5)
print("Columns:", list(df_imp.columns))
print(df_imp.head())

# Check presence of watch_percentage and rating columns
cols = set(c.lower() for c in df_imp.columns)
print("Has watch_percentage?", "watch_percentage" in cols)
print("Has rating?", "rating" in cols)


Implicit CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\raw\mars\implicit_ratings_en.csv
Columns: ['item_id', 'user_id', 'created_at']
   item_id  user_id           created_at
0      510     4287  2016-11-02 20:59:23
1      510     4287  2017-06-20 00:33:46
2      511     4287  2017-06-20 00:44:42
3      512     4287  2017-06-20 00:52:11
4      513     4287  2017-06-20 00:54:41
Has watch_percentage? False
Has rating? False


Column mapping to canonical event schema (auto-detect + override)

In [11]:
# [CELL 02-06] Map raw columns -> canonical schema (EXPLICIT ONLY)

raw_cols = con.execute(f"SELECT * FROM {RAW_VIEW} LIMIT 1").df().columns.tolist()
raw_cols_lower = [c.lower() for c in raw_cols]
print("[02-06] Raw columns:", raw_cols)

def first_match(candidates):
    for cand in candidates:
        if cand in raw_cols_lower:
            return raw_cols[raw_cols_lower.index(cand)]
    return None

COL_USER = first_match(["user_id", "userid", "user", "uid"])
COL_ITEM = first_match(["item_id", "item", "course_id", "cid"])
COL_TIME = first_match(["created_at", "timestamp", "time", "event_time", "ts"])
COL_RATING = first_match(["rating"])

print("[02-06] Detected mapping:")
print("  COL_USER  =", COL_USER)
print("  COL_ITEM  =", COL_ITEM)
print("  COL_TIME  =", COL_TIME)
print("  COL_RATING=", COL_RATING)

missing = [k for k,v in {"COL_USER": COL_USER, "COL_ITEM": COL_ITEM, "COL_TIME": COL_TIME, "COL_RATING": COL_RATING}.items() if v is None]
if missing:
    raise ValueError("Missing required columns: " + ", ".join(missing))


[02-06] Raw columns: ['user_id', 'item_id', 'watch_percentage', 'created_at', 'rating']
[02-06] Detected mapping:
  COL_USER  = user_id
  COL_ITEM  = item_id
  COL_TIME  = created_at
  COL_RATING= rating


Build a canonical events_target view (types + null filtering; no sessions)

In [24]:
# [CELL 02-07] Create canonical view (EXPLICIT-ONLY TARGET)
# Decision (Option A): Target dataset uses ONLY explicit_ratings_en.csv.
# Therefore:
#   signal_type is always 'explicit_rating'
#   value is normalized rating (assumed 0..10 -> divide by 10)
# Notes:
# - No implicit_ratings_en.csv is included in explicit-only mode.
# - No watch_percentage is expected here.

def qident(name: str) -> str:
    return '"' + name.replace('"', '""') + '"'

CANON_VIEW = "events_target"

u = qident(COL_USER)
i = qident(COL_ITEM)
t = qident(COL_TIME)
r = qident(COL_RATING)

print("\n" + "="*70)
print("02-07 CANONICAL VIEW (EXPLICIT-ONLY MODE)")
print("="*70)
print("[02-07] Using ONLY explicit_ratings_en.csv.")
print("[02-07] Canonical schema: user_id, item_id, timestamp, signal_type, value")
print("[02-07] signal_type='explicit_rating' for all rows.")
print("[02-07] value = rating / 10.0 (rating scale sanity check below).")

# rating sanity check (raw)
rating_check = con.execute(f"""
SELECT
  COUNT(*) AS n_rows,
  COUNT(*) FILTER (WHERE try_cast({r} AS DOUBLE) IS NULL) AS n_bad_rating,
  MIN(try_cast({r} AS DOUBLE)) AS min_rating,
  MAX(try_cast({r} AS DOUBLE)) AS max_rating
FROM {RAW_VIEW};
""").df()

print("[02-07] Raw rating sanity check:")
display(rating_check)

# Create canonical view
con.execute(
    f"""
    CREATE OR REPLACE VIEW {CANON_VIEW} AS
    SELECT
      CAST({u} AS VARCHAR) AS user_id,
      CAST({i} AS VARCHAR) AS item_id,
      CAST({t} AS TIMESTAMP) AS timestamp,
      'explicit_rating' AS signal_type,
      (try_cast({r} AS DOUBLE) / 10.0) AS value
    FROM {RAW_VIEW}
    WHERE {u} IS NOT NULL
      AND {i} IS NOT NULL
      AND {t} IS NOT NULL
      AND try_cast({r} AS DOUBLE) IS NOT NULL
    """
)

canon_sample = con.execute(f"SELECT * FROM {CANON_VIEW} LIMIT 10").df()
print("[02-07] Canonical sample:")
display(canon_sample)

bad_ts = con.execute(f"SELECT COUNT(*) AS n_bad_ts FROM {CANON_VIEW} WHERE timestamp IS NULL").df().iloc[0]["n_bad_ts"]
print("[02-07] Bad timestamp rows (timestamp is NULL after cast):", int(bad_ts))
if bad_ts > 0:
    raise ValueError("Timestamp casting produced NULLs. Check created_at format.")

# quick distribution of raw rating values (helps confirm scale)
rating_dist = con.execute(f"""
SELECT
  try_cast({r} AS DOUBLE) AS rating,
  COUNT(*) AS n
FROM {RAW_VIEW}
WHERE try_cast({r} AS DOUBLE) IS NOT NULL
GROUP BY 1
ORDER BY n DESC
LIMIT 20;
""").df()

print("[02-07] Top rating values (raw):")
display(rating_dist)



02-07 CANONICAL VIEW (EXPLICIT-ONLY MODE)
[02-07] Using ONLY explicit_ratings_en.csv.
[02-07] Canonical schema: user_id, item_id, timestamp, signal_type, value
[02-07] signal_type='explicit_rating' for all rows.
[02-07] value = rating / 10.0 (rating scale sanity check below).
[02-07] Raw rating sanity check:


,n_rows,n_bad_rating,min_rating,max_rating
0,3659,0,1.0,10.0


[02-07] Canonical sample:


,user_id,item_id,timestamp,signal_type,value
0,224557,510,2018-09-28 16:18:29,explicit_rating,1.0
1,224557,615,2018-09-28 16:22:22,explicit_rating,1.0
2,224557,7680,2018-09-28 16:23:34,explicit_rating,1.0
3,224293,510,2018-09-28 17:20:30,explicit_rating,1.0
4,224293,515,2018-09-28 17:40:02,explicit_rating,1.0
5,224293,516,2018-09-28 17:41:45,explicit_rating,0.4
6,224442,510,2018-10-01 20:11:35,explicit_rating,1.0
7,196845,510,2018-10-06 13:12:52,explicit_rating,0.1
8,265845,510,2018-10-10 21:17:26,explicit_rating,1.0
9,224470,510,2018-10-11 19:59:49,explicit_rating,1.0


[02-07] Bad timestamp rows (timestamp is NULL after cast): 0
[02-07] Top rating values (raw):


,rating,n
0,10.0,2747
1,1.0,200
2,2.0,135
3,4.0,105
4,9.0,96
5,3.0,90
6,8.0,79
7,6.0,74
8,7.0,72
9,5.0,61


Core EDA (counts, time range, top actions, quantiles)

In [13]:
# [CELL 02-08] EDA metrics (EXPLICIT ONLY)

counts = con.execute(
    f"""
    SELECT
      COUNT(*) AS n_events,
      COUNT(DISTINCT user_id) AS n_users,
      COUNT(DISTINCT item_id) AS n_items,
      MIN(timestamp) AS min_ts,
      MAX(timestamp) AS max_ts
    FROM {CANON_VIEW};
    """
).df()
print("[02-08] Counts overview:")
display(counts)

signal_dist = con.execute(
    f"""
    SELECT signal_type, COUNT(*) AS n
    FROM {CANON_VIEW}
    GROUP BY 1
    ORDER BY n DESC;
    """
).df()
print("[02-08] signal_type distribution:")
display(signal_dist)

events_per_user = con.execute(
    f"""
    WITH per_user AS (
      SELECT user_id, COUNT(*) AS cnt
      FROM {CANON_VIEW}
      GROUP BY 1
    )
    SELECT
      approx_quantile(cnt, [0.5, 0.9, 0.99]) AS q,
      MIN(cnt) AS min_cnt,
      MAX(cnt) AS max_cnt,
      AVG(cnt) AS avg_cnt
    FROM per_user;
    """
).df()
print("[02-08] Events-per-user quantiles + min/max/avg:")
display(events_per_user)

value_stats = con.execute(
    f"""
    SELECT
      COUNT(*) AS n,
      MIN(value) AS min_value,
      MAX(value) AS max_value,
      AVG(value) AS avg_value
    FROM {CANON_VIEW};
    """
).df()
print("[02-08] Value stats:")
display(value_stats)


[02-08] Counts overview:


,n_events,n_users,n_items,min_ts,max_ts
0,3659,822,776,2018-09-28 14:38:15,2021-09-20 16:26:06


[02-08] signal_type distribution:


,signal_type,n
0,explicit_rating,3659


[02-08] Events-per-user quantiles + min/max/avg:


,q,min_cnt,max_cnt,avg_cnt
0,"[2, 8, 59]",1,134,4.451338


[02-08] Value stats:


,n,min_value,max_value,avg_value
0,3659,0.1,1.0,0.857584


USER SEQUENCE CHARACTERISTICS

In [14]:
print("\n" + "="*70)
print("USER SEQUENCE CHARACTERISTICS")
print("="*70)

items_per_user = con.execute(f"""
    WITH per_user AS (
      SELECT 
        user_id, 
        COUNT(DISTINCT item_id) AS n_items,
        COUNT(*) AS n_events,
        MIN(timestamp) AS first_event,
        MAX(timestamp) AS last_event
      FROM {CANON_VIEW}
      GROUP BY user_id
    )
    SELECT
      approx_quantile(n_items,  [0.5, 0.9, 0.99]) AS items_quantiles,
      approx_quantile(n_events, [0.5, 0.9, 0.99]) AS events_quantiles,
      MIN(n_items) AS min_items,
      MAX(n_items) AS max_items,
      AVG(n_items) AS avg_items,
      AVG(EXTRACT(EPOCH FROM (last_event - first_event)) / 86400.0) AS avg_span_days
    FROM per_user
""").df()

print("\nItems per user:")
display(items_per_user)

sufficient_history = con.execute(f"""
    WITH per_user AS (
      SELECT user_id, COUNT(*) AS n_events
      FROM {CANON_VIEW}
      GROUP BY user_id
    )
    SELECT
      COUNT(*) AS total_users,
      SUM(CASE WHEN n_events >= 2  THEN 1 ELSE 0 END) AS users_2plus,
      SUM(CASE WHEN n_events >= 5  THEN 1 ELSE 0 END) AS users_5plus,
      SUM(CASE WHEN n_events >= 10 THEN 1 ELSE 0 END) AS users_10plus,
      SUM(CASE WHEN n_events >= 20 THEN 1 ELSE 0 END) AS users_20plus
    FROM per_user
""").df()

print("\nUsers by sequence length:")
display(sufficient_history)

row = sufficient_history.iloc[0]
total = int(row["total_users"])
def pct(x): 
    return (float(x) / total * 100.0) if total > 0 else 0.0

print(f"\n  ✅ {int(row['users_2plus']):,} users ({pct(row['users_2plus']):.1f}%) have >=2 events")
print(f"  ✅ {int(row['users_5plus']):,} users ({pct(row['users_5plus']):.1f}%) have >=5 events")
print(f"  ✅ {int(row['users_10plus']):,} users ({pct(row['users_10plus']):.1f}%) have >=10 events")



USER SEQUENCE CHARACTERISTICS

Items per user:


,items_quantiles,events_quantiles,min_items,max_items,avg_items,avg_span_days
0,"[2, 8, 59]","[2, 8, 59]",1,134,4.446472,30.560771



Users by sequence length:


,total_users,users_2plus,users_5plus,users_10plus,users_20plus
0,822,426.0,164.0,73.0,31.0



  ✅ 426 users (51.8%) have >=2 events
  ✅ 164 users (20.0%) have >=5 events
  ✅ 73 users (8.9%) have >=10 events


ITEM (COURSE) CHARACTERISTICS

In [15]:
print("\n" + "="*70)
print("ITEM (COURSE) CHARACTERISTICS")
print("="*70)

events_per_item = con.execute(f"""
    WITH per_item AS (
      SELECT 
        item_id,
        COUNT(*) AS n_events,
        COUNT(DISTINCT user_id) AS n_users,
        MIN(timestamp) AS first_event,
        MAX(timestamp) AS last_event
      FROM {CANON_VIEW}
      GROUP BY item_id
    )
    SELECT
      COUNT(*) AS total_items,
      approx_quantile(n_events, [0.5, 0.9, 0.99]) AS events_quantiles,
      approx_quantile(n_users,  [0.5, 0.9, 0.99]) AS users_quantiles,
      MIN(n_events) AS min_events,
      MAX(n_events) AS max_events,
      AVG(n_events) AS avg_events,
      SUM(CASE WHEN n_events = 1 THEN 1 ELSE 0 END) AS items_single_event,
      SUM(CASE WHEN n_events <= 5 THEN 1 ELSE 0 END) AS items_cold_5
    FROM per_item
""").df()

print("\nEvents per item (course):")
display(events_per_item)

top_items = con.execute(f"""
    SELECT 
      item_id,
      COUNT(*) AS n_events,
      COUNT(DISTINCT user_id) AS n_users,
      AVG(value) AS avg_value
    FROM {CANON_VIEW}
    GROUP BY item_id
    ORDER BY n_events DESC
    LIMIT 10
""").df()

print("\nTop 10 most popular courses:")
display(top_items)



ITEM (COURSE) CHARACTERISTICS

Events per item (course):


,total_items,events_quantiles,users_quantiles,min_events,max_events,avg_events,items_single_event,items_cold_5
0,776,"[2, 8, 35]","[2, 8, 35]",1,179,4.715206,226.0,607.0



Top 10 most popular courses:


,item_id,n_events,n_users,avg_value
0,510,179,179,0.886592
1,43457,109,109,0.859633
2,512,78,78,0.751282
3,7626,76,76,0.673684
4,733,48,48,0.833333
5,545,44,44,0.736364
6,16366,42,42,0.878571
7,43458,35,35,0.925714
8,32022,33,33,0.754545
9,183716,31,30,0.761290


TIMESTAMP QUALITY CHECKS

In [16]:
print("\n" + "="*70)
print("TIMESTAMP QUALITY CHECKS")
print("="*70)

future_check = con.execute(f"""
    SELECT COUNT(*) AS n_future
    FROM {CANON_VIEW}
    WHERE timestamp > CURRENT_TIMESTAMP
""").df()
print(f"\nFuture timestamps: {int(future_check.iloc[0]['n_future'])}")

dup_check = con.execute(f"""
    WITH dups AS (
      SELECT user_id, item_id, timestamp, COUNT(*) AS cnt
      FROM {CANON_VIEW}
      GROUP BY 1, 2, 3
      HAVING COUNT(*) > 1
    )
    SELECT COUNT(*) AS n_dup_groups, COALESCE(SUM(cnt), 0) AS n_dup_rows
    FROM dups
""").df()

print("\nDuplicate (user, item, timestamp) tuples:")
display(dup_check)

order_check = con.execute(f"""
    WITH ordered AS (
      SELECT 
        user_id,
        timestamp,
        LAG(timestamp) OVER (PARTITION BY user_id ORDER BY timestamp) AS prev_ts
      FROM {CANON_VIEW}
    )
    SELECT COUNT(*) AS n_out_of_order
    FROM ordered
    WHERE prev_ts IS NOT NULL AND timestamp < prev_ts
""").df()

n_out = int(order_check.iloc[0]["n_out_of_order"])
print(f"\nOut-of-order events: {n_out}")
if n_out > 0:
    print("  ⚠️  WARNING: Some timestamps are not monotonic within users!")



TIMESTAMP QUALITY CHECKS

Future timestamps: 0

Duplicate (user, item, timestamp) tuples:


,n_dup_groups,n_dup_rows
0,4,8.0



Out-of-order events: 0


Daily volume series + simple plots (saved under reports/)

SPARSITY ANALYSIS

In [17]:
print("\n" + "="*70)
print("SPARSITY ANALYSIS")
print("="*70)

sparsity = con.execute(f"""
    WITH stats AS (
      SELECT 
        COUNT(*) AS n_events,
        COUNT(DISTINCT user_id) AS n_users,
        COUNT(DISTINCT item_id) AS n_items
      FROM {CANON_VIEW}
    )
    SELECT
      n_events,
      n_users,
      n_items,
      (n_users * n_items) AS matrix_size,
      ROUND(100.0 * n_events / NULLIF((n_users * n_items), 0), 6) AS density_pct,
      ROUND(100.0 - (100.0 * n_events / NULLIF((n_users * n_items), 0)), 6) AS sparsity_pct
    FROM stats
""").df()

print("\nUser-Item matrix sparsity:")
display(sparsity)

row = sparsity.iloc[0]
print(f"\n  Matrix size: {int(row['n_users']):,} users × {int(row['n_items']):,} items = {int(row['matrix_size']):,} cells")
print(f"  Observed: {int(row['n_events']):,} events")
print(f"  Density: {float(row['density_pct']):.6f}%")
print(f"  Sparsity: {float(row['sparsity_pct']):.6f}%")



SPARSITY ANALYSIS

User-Item matrix sparsity:


,n_events,n_users,n_items,matrix_size,density_pct,sparsity_pct
0,3659,822,776,637872,0.573626,99.426374



  Matrix size: 822 users × 776 items = 637,872 cells
  Observed: 3,659 events
  Density: 0.573626%
  Sparsity: 99.426374%


DATA PROVENANCE & LIMITATIONS (text only)

In [18]:
print("\n" + "="*70)
print("DATA PROVENANCE & LIMITATIONS")
print("="*70)

provenance = f"""
Dataset: MARS (Target MOOC)
Source: [CITATION TBD]
License: [TBD]

Known characteristics from this run:
  - Time period: {counts.iloc[0]['min_ts']} to {counts.iloc[0]['max_ts']}
  - Signals: explicit_rating + implicit_watch
  - Schema: (user_id, item_id(course), timestamp, signal_type, value)

Limitations (as observed / structural):
  1) Event-level interactions (not session-labeled at source)
  2) No user demographics/context features in these CSVs
  3) No course metadata used here (items_en.csv exists but not joined in EDA)
  4) Sparsity: see SPARSITY ANALYSIS cell output
  5) Sequence lengths: see USER SEQUENCE CHARACTERISTICS output

TODO before publication:
  - [ ] Find and cite original MARS dataset source
  - [ ] Verify license for academic use
  - [ ] Confirm rating scale and watch_percentage semantics
"""
print(provenance)

# Save provenance text for the run
prov_path = RUN_REPORT_DIR / "provenance_and_limitations.txt"
prov_path.write_text(provenance.strip() + "\n", encoding="utf-8")
print("[02-08E] Saved:", prov_path)



DATA PROVENANCE & LIMITATIONS

Dataset: MARS (Target MOOC)
Source: [CITATION TBD]
License: [TBD]

Known characteristics from this run:
  - Time period: 2018-09-28 14:38:15 to 2021-09-20 16:26:06
  - Signals: explicit_rating + implicit_watch
  - Schema: (user_id, item_id(course), timestamp, signal_type, value)

Limitations (as observed / structural):
  1) Event-level interactions (not session-labeled at source)
  2) No user demographics/context features in these CSVs
  3) No course metadata used here (items_en.csv exists but not joined in EDA)
  4) Sparsity: see SPARSITY ANALYSIS cell output
  5) Sequence lengths: see USER SEQUENCE CHARACTERISTICS output

TODO before publication:
  - [ ] Find and cite original MARS dataset source
  - [ ] Verify license for academic use
  - [ ] Confirm rating scale and watch_percentage semantics

[02-08E] Saved: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\provenance_and_limitations.txt


In [19]:
# [CELL 02-09] Daily volume series + plots

daily = con.execute(
    f"""
    SELECT
      date_trunc('day', timestamp) AS day,
      COUNT(*) AS n
    FROM {CANON_VIEW}
    GROUP BY 1
    ORDER BY 1;
    """
).df()

print("[02-09] Daily rows:", daily.shape)
display(daily.head())

# Save tables
save_df_csv(RUN_REPORT_DIR / "counts_overview.csv", counts)
save_df_csv(RUN_REPORT_DIR / "signal_type_dist.csv", signal_dist)
save_df_csv(RUN_REPORT_DIR / "events_per_user.csv", events_per_user)
save_df_csv(RUN_REPORT_DIR / "value_stats.csv", value_stats)
save_df_csv(RUN_REPORT_DIR / "daily_event_volume.csv", daily)

# Plot (matplotlib only; no fixed colors)
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(pd.to_datetime(daily["day"]), daily["n"])
plt.title("MARS target daily event volume")
plt.xlabel("Day")
plt.ylabel("Events")
plt.tight_layout()

plot_path = RUN_REPORT_DIR / "daily_event_volume.png"
plt.savefig(plot_path, dpi=150)
plt.close()
print("[02-09] Saved plot:", plot_path)


[02-09] Daily rows: (595, 2)


,day,n
0,2018-09-28,44
1,2018-09-29,2
2,2018-10-01,1
3,2018-10-02,2
4,2018-10-04,31


[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\counts_overview.csv | shape: (1, 5)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\signal_type_dist.csv | shape: (1, 2)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\events_per_user.csv | shape: (1, 4)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\value_stats.csv | shape: (1, 4)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\daily_event_volume.csv | shape: (595, 2)
[02-09] Saved plot: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\daily_event_volume.png


In [20]:
save_df_csv(RUN_REPORT_DIR / "user_items_per_user.csv", items_per_user)
save_df_csv(RUN_REPORT_DIR / "user_sufficient_history.csv", sufficient_history)
save_df_csv(RUN_REPORT_DIR / "item_events_per_item.csv", events_per_item)
save_df_csv(RUN_REPORT_DIR / "item_top_10.csv", top_items)
save_df_csv(RUN_REPORT_DIR / "timestamp_future_check.csv", future_check)
save_df_csv(RUN_REPORT_DIR / "timestamp_dup_check.csv", dup_check)
save_df_csv(RUN_REPORT_DIR / "timestamp_order_check.csv", order_check)
save_df_csv(RUN_REPORT_DIR / "sparsity.csv", sparsity)


[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\user_items_per_user.csv | shape: (1, 6)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\user_sufficient_history.csv | shape: (1, 5)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\item_events_per_item.csv | shape: (1, 8)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\item_top_10.csv | shape: (10, 4)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\timestamp_future_check.csv | shape: (1, 1)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_131137\timestamp_dup_check.csv | shape: (1, 2)
[02-02] Wrote CSV: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_tar

Export canonical parquet for downstream normalization (Notebook 03)

In [21]:
# [CELL 02-10] Export canonical target events parquet (EXPLICIT ONLY)

OUT_EVENTS = TARGET_PROCESSED_DIR / "mars_events_canonical.parquet"

con.execute(
    f"""
    COPY (
      SELECT user_id, item_id, timestamp, signal_type, value
      FROM {CANON_VIEW}
    )
    TO '{str(OUT_EVENTS)}'
    (FORMAT PARQUET);
    """
)

print("[02-10] Wrote canonical target events parquet:", OUT_EVENTS)


[02-10] Wrote canonical target events parquet: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\mars\mars_events_canonical.parquet


Optional: define KEEP_ACTIONS for target + export filtered interactions

In [22]:
# [CELL 02-11] Explicit-only setup: no implicit exports

OUT_EXPLICIT = TARGET_PROCESSED_DIR / "mars_events_explicit_only.parquet"

con.execute(
    f"""
    COPY (
      SELECT user_id, item_id, timestamp, signal_type, value
      FROM {CANON_VIEW}
    )
    TO '{str(OUT_EXPLICIT)}'
    (FORMAT PARQUET);
    """
)

print("[02-11] Wrote:", OUT_EXPLICIT)
print("[02-11] Explicit-only mode: skipping implicit exports.")


[02-11] Wrote: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\mars\mars_events_explicit_only.parquet
[02-11] Explicit-only mode: skipping implicit exports.


Save dataset metadata for reproducibility

In [22]:
# [CELL 02-12] Save dataset metadata for reproducibility (JSON-safe)

import numpy as np

def json_safe(x):
    """Convert numpy/pandas types into JSON-serializable Python types."""
    # numpy scalar -> python scalar
    if isinstance(x, (np.integer, np.int64, np.int32)):
        return int(x)
    if isinstance(x, (np.floating, np.float64, np.float32)):
        return float(x)
    if isinstance(x, (np.bool_,)):
        return bool(x)

    # numpy array -> list
    if isinstance(x, np.ndarray):
        return x.tolist()

    # pandas Timestamp / datetime -> str
    if isinstance(x, (pd.Timestamp, datetime)):
        return str(x)

    # dict -> recurse
    if isinstance(x, dict):
        return {k: json_safe(v) for k, v in x.items()}

    # list/tuple -> recurse
    if isinstance(x, (list, tuple)):
        return [json_safe(v) for v in x]

    # fallback: keep primitives / stringify unknowns
    if x is None or isinstance(x, (str, int, float, bool)):
        return x
    return str(x)


counts_row = counts.iloc[0].to_dict()

meta = {
    "dataset": "MARS (target)",
    "run_tag": RUN_TAG,
    "canonical_schema": ["user_id", "item_id", "timestamp", "signal_type", "value"],
    "notes": [
        "In MARS, item_id is treated as course_id (items are courses).",
        "Hybrid signals are normalized: rating/10, watch_percentage/100.",
        "No session analysis or sessionization in this notebook."
    ],
    "raw_files": {
        "explicit_ratings_en.csv": str(explicit_csv),
        "implicit_ratings_en.csv": str(implicit_csv),
        "items_en.csv": str(items_csv) if "items_csv" in globals() else None,
        "users_en.csv": str(users_csv) if "users_csv" in globals() else None,
    },
    "mapping": {
        "COL_USER": COL_USER,
        "COL_ITEM": COL_ITEM,
        "COL_TIME": COL_TIME,
        "COL_RATING": COL_RATING,
        "COL_WATCH": COL_WATCH,
    },
    "counts": {
        "n_events": counts_row["n_events"],
        "n_users": counts_row["n_users"],
        "n_items": counts_row["n_items"],
        "min_ts": counts_row["min_ts"],
        "max_ts": counts_row["max_ts"],
    },
    "exports": {
        "mars_events_canonical": str(OUT_EVENTS),
        "mars_events_explicit_only": str(OUT_EXPLICIT) if "OUT_EXPLICIT" in globals() else None,
        "mars_events_implicit_only": str(OUT_IMPLICIT) if "OUT_IMPLICIT" in globals() else None,
    },
    "reports_dir": str(RUN_REPORT_DIR),
}

# ---------- Extra EDA (only if cells exist) ----------
extra_eda = {}

if "items_per_user" in globals():
    extra_eda["items_per_user"] = items_per_user.to_dict(orient="records")[0]

if "sufficient_history" in globals():
    extra_eda["sufficient_history"] = sufficient_history.to_dict(orient="records")[0]

if "events_per_item" in globals():
    extra_eda["events_per_item"] = events_per_item.to_dict(orient="records")[0]

if "top_items" in globals():
    extra_eda["top_items"] = top_items.to_dict(orient="records")

if "future_check" in globals() or "dup_check" in globals() or "order_check" in globals():
    extra_eda["timestamp_quality"] = {
        "n_future": future_check.iloc[0]["n_future"] if "future_check" in globals() else None,
        "dup_check": dup_check.to_dict(orient="records")[0] if "dup_check" in globals() else None,
        "n_out_of_order": order_check.iloc[0]["n_out_of_order"] if "order_check" in globals() else None,
    }

if "sparsity" in globals():
    extra_eda["sparsity"] = sparsity.to_dict(orient="records")[0]

meta["extra_eda"] = extra_eda

# ---------- Provenance / limitations ----------
meta["provenance"] = {
    "dataset_name": "MARS",
    "citation": "[CITATION TBD]",
    "license": "[TBD]",
    "limitations": [
        "Event-level interactions (not session-labeled at source).",
        "No user demographics/context features in these CSVs.",
        "No course metadata joined in this notebook (items_en.csv exists but not joined).",
        "Sparse user-item matrix (see sparsity in extra_eda).",
        "Short sequences for many users (see sufficient_history in extra_eda).",
    ],
    "notes": "See reports/.../provenance_and_limitations.txt for the run text if generated.",
}

# Make everything JSON serializable
meta = json_safe(meta)

# Save into processed (so Notebook 03 can rely on it) and also into the run report folder
save_json(TARGET_PROCESSED_DIR / "dataset_metadata.json", meta)
save_json(RUN_REPORT_DIR / "dataset_metadata.json", meta)

print("[02-12] Done. Next notebook is 03_schema_normalization.ipynb")


[02-02] Wrote JSON: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\data\processed\mars\dataset_metadata.json
[02-02] Wrote JSON: D:\00_DS-ML-Workspace\mooc-coldstart-session-meta\reports\02_eda_target_mooc\20251229_115541\dataset_metadata.json
[02-12] Done. Next notebook is 03_schema_normalization.ipynb
